In [2]:
from dotenv import load_dotenv
load_dotenv()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_anthropic import ChatAnthropic
from src.path import DATA_DIR
from langchain_community.embeddings import HuggingFaceEmbeddings
from pathlib import Path

# Load Document

In [3]:
# 문서 로드 및 분할 설정
TAX_FPATH1 = DATA_DIR / "context (1).docx"
loader = Docx2txtLoader(TAX_FPATH1)
document = loader.load()

In [4]:
# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap =30
)

In [5]:
loader = Docx2txtLoader(TAX_FPATH1)
documents = loader.load_and_split(text_splitter=text_splitter)

# Documnet Embdding

In [ ]:
# 임베딩 모델 선정
embedding = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",
    model_kwargs={"device": "cpu"}
)

# 벡터 데이터베이스(DB) 디렉터리 설정
DB_DIR = DATA_DIR / "KURE_DB"
if not DB_DIR.exists():
    DB_DIR.mkdir(parents=True)

# Chroma 벡터 DB 생성
db = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=DB_DIR
)

print("✅ KURE-v1 임베딩 기반 Chroma DB 생성 완료")

C:\Users\jong8\AppData\Local\Temp\ipykernel_13888\4213289187.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


✅ KURE-v1 임베딩 기반 Chroma DB 생성 완료


# Search

In [7]:
raw_query = '비수도권에 사는 청년'

In [8]:
retrieved_docs = db.similarity_search(raw_query, k = 2)

In [9]:
retrieved_docs

[Document(id='b3794ede-4759-4dcf-81cc-84a2433a60e0', metadata={'source': 'c:\\Users\\jong8\\Desktop\\langchain\\data\\context (1).docx'}, page_content='생산 활동이나 육체노동을 통해 재화나 서비스를 만드는 역할로, 차량 운전자, 현장직, 생산직 등이 있습니다. “기능직”은 특정 기술이나 숙련된 손기술을 활용하는 직업으로, 기술직, 제빵업, 목수, 전기공, 정비사, 배관공 등이 있으며, 이 외에도 토지나 자원을 활용하는 “농업/임업/축산업/광업/수산업”과 부동산 등의 자산을 활용하는 “임대업”이 있습니다. 비경제활동인구 또는 특수 직업군으로는 “중/고등학생”, “대학생/대학원생”, “전업주부”, “퇴직/연금생활자”가 있으며, 이 외에도 “군인”, “공공안전”, “문화콘텐츠”,'),
 Document(id='de4d9a87-9a34-4633-9142-a5946db226c8', metadata={'source': 'c:\\Users\\jong8\\Desktop\\langchain\\data\\context (1).docx'}, page_content='생산 활동이나 육체노동을 통해 재화나 서비스를 만드는 역할로, 차량 운전자, 현장직, 생산직 등이 있습니다. “기능직”은 특정 기술이나 숙련된 손기술을 활용하는 직업으로, 기술직, 제빵업, 목수, 전기공, 정비사, 배관공 등이 있으며, 이 외에도 토지나 자원을 활용하는 “농업/임업/축산업/광업/수산업”과 부동산 등의 자산을 활용하는 “임대업”이 있습니다. 비경제활동인구 또는 특수 직업군으로는 “중/고등학생”, “대학생/대학원생”, “전업주부”, “퇴직/연금생활자”가 있으며, 이 외에도 “군인”, “공공안전”, “문화콘텐츠”,')]

# Load Saved VectorDB

In [10]:
DB_DIR = Path("NEW_DB")

embedding = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",
    model_kwargs={"device": "cpu"}
   )

db = Chroma(persist_directory=DB_DIR, embedding_function = embedding)

# Simple Method

In [11]:
user_input = '비수도권에 사는 청년'

In [ ]:
retrieved_docs = db.similarity_search(user_input, k = 2) # 유사 문서 검색

In [ ]:
# LLM 프롬프트 설정
message = f"""
역할: 사용자 입력으로 SQL (WHERE절)과 오피니언/해시태그를 동시 생성.
---
## 1. SQL (meta query) 규칙
**출력:** `SELECT * FROM panel_records WHERE ...;` (조건 없으면 WHERE 절 생략)
**형식:** 필드 중 값이 있는 것만 AND 연결. 문자열 값은 반드시 작은따옴표(')로 묶음.
**필드:** gender, birth, region, subregion, married, nchild, famsize, education_level, job, work, p_income, h_income, owned_products, phone_brand, phone_model, car_ownship, car_manufacturer, car_model, ever_smoked, brand_smoked, brand_smoked_ETC, ever_esmoked, ever_smoked_brand_ETC, ever_alcohol, ever_smoked_ETC, p_company.
**특수 규칙:**
1.  **gender/married/nchild/famsize:** '여/여자'→'F'...,'결혼'→`married='기혼'`. 자녀있음→`nchild > 0`. `nchild`는 int. * `famsize`: **['1명(혼자 거주)', '2명'...]값만 사용 (범위는 `IN`). * `p_income`: '월 N00~N99만원'(100단위 문자열). 저소득(~199) 등은 해당 구간들을 `IN`으로 연결.
2.  **휴대폰:** `phone_model LIKE '%값%'` 사용. `car_model`은 동등비교(`=`)만.    
3.  **보유제품(owned_products):** 제품군 언급 시 그룹 내 모든 세부 제품을 OR로 연결 (`owned_products LIKE '%X%' OR ...`).

## 2. 직군(Work/Job) 데이터 (우선순위: Work > Job)
**[Rule A] 직무(Work) - 100% 일치 시 사용 (`=`)**
> ['경영/인사/총무/사무'...]

**[Rule B] 직업(Job) - 의미상 가장 가까운 값 선택 (`=`)**
Rule A 실패 시, 아래 리스트 중 사용자의 직업이 포함되거나 가장 유사한 값을 선택.
* **주의:** 포괄적 단어는 모든 work를 나열하지 말고, **무조건 하나로만 매핑**.
* **매핑 예시:** (직장인/회사원/공무원 → '사무직'...)

## 3. 오피니언 (opinion) 및 해시태그 규칙
**판정:** 사용자의 선호/의견/감정/가치/취향/습관/루틴/빈도/행동 의도가 드러나면 **존재**. 단순 사실(거주지, 나이, 직업 등)만 있으면 **부재**.
**존재 시:**
1.  **opinion:** 사용자 입력 반영하여 군더더기 없이 한 문장으로 요약.
2.  **main/sub:** 아래 목록에서 각각 1개 선택. 다른 단어 추가/변형/순서 변경 금지.
**부재 시:** `opinion`, `main`, `sub` 모두 "-" 처리.
**해시태그 목록:**
* `#main "여가와 문화"` / `- "여행 이외의 모든 오프라인 문화생활"`, `- "여행 기반 오프라인 문화생활"`
* `#main "일상 요소"` / `- "경험 추억 등 과거와 관련된 행동"`, `- "환경과 관련된 행동"`, `- "일상적으로 반복하는 행동"`
* `#main "스타일 외모"` / `- "패션 관련 뷰티"`, `- "패션 외적인 뷰티"`
* `#main "기술 및 정보"` / `- "디지털 도구 활용"`
* `#main "소비와 재정"` / `- "소비를 통해 이득을 취하는 경우"`, `- "소비를 통해 가치관을 표현"`
* `#main "건강 웰빙"` / `- "신체적 건강"`, `- "신체적·심적인 건강"` 

## 4. 최종 출력 규칙 (절대 준수)
**출력:** "sql","opinion","main","sub" 4가지 키만 JSON 형식이 아닌 그대로 출력.
**부재:** "-"으로만 처리. 선정 이유 및 연산 과정 출력 금지.

**[출력예시]**
"sql": "SELECT * FROM panel_records WHERE region = '서울' AND job = '대학생/대학원생' AND ever_smoked = '담배를 피워본 적이 없다';",
"opinion": "환경문제에 관심이 많다"
"main" : "일상 요소"
"sub" : "환경과 관련된 행동"

user_input:
{user_input}

참고:
{retrieved_docs}
""".strip()

In [ ]:
# Anthropic LLM 설정 및 호출
llm_consistent= ChatAnthropic(
    model='claude-opus-4-20250514',
    anthropic_api_key="<YOUR_ANTHROPIC_API_KEY>", #API 키 삭제
    temperature=0,
    max_tokens=1000,
)

In [15]:
response = llm_consistent.invoke(message)

In [16]:
print(response.content)

"sql": "SELECT * FROM panel_records WHERE region IN ('부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주') AND birth >= 1994;",
"opinion": "-"
"main" : "-"
"sub" : "-"
